In [135]:
import numpy as np
import scipy

In [195]:
def return_calculate(price,method):
    names=price.columns
    num=len(names)
    names_2=names[names!='Date']

    if num==len(names_2):
        print("")
    else:
        num=num-1

    pos=np.where(names!='Date')[0]
    row_num=price.shape[0]
    out=np.zeros([row_num-1,num])
    for i in range(num):
        temp=np.array(price.iloc[:,pos[i]])

        if method.upper()=='DISCRETE':
            out[:,i]=temp[1:]/temp[0:-1]-1
        elif method.upper()=='LOG':
            out[:,i]=np.log(temp[1:])-np.log(temp[0:-1])
        else:
            print('method must in Log or Discrete')

    out=pd.DataFrame(out)
    out.index=price.index[1:]
    out.columns=price.columns[pos]

    return out

In [198]:
ff = pd.read_csv('F-F_Research_Data_Factors_daily.csv', parse_dates=['Date']).set_index('Date')
fm = pd.read_csv('F-F_Momentum_Factor_daily.csv', parse_dates=['Date']).set_index('Date')
ff4 = ff.join(fm, how='right') / 100 

In [197]:
prices = pd.read_csv('DailyPrices.csv', parse_dates=['Date']).set_index('Date')
rt=return_calculate(prices,method='Discrete')
stocks = ['AAPL', 'META', 'UNH', 'MA',  
          'MSFT' ,'NVDA', 'HD', 'PFE',  
          'AMZN' ,'BRK-B', 'PG', 'XOM',  
          'TSLA' ,'JPM' ,'V', 'DIS',  
          'GOOGL', 'JNJ', 'BAC', 'CSCO']
factors = ['Mkt-RF', 'SMB', 'HML', 'mom']
dataset = rt[stocks].join(ff_4)
dataset = dataset.loc['2022-2-14':'2023-1-31']
nan_mask = np.isnan(dataset)
inf_mask = np.isinf(dataset)
nan_rows, nan_cols = np.where(nan_mask)
inf_rows, inf_cols = np.where(inf_mask)
if len(nan_rows) > 0:
    print(f"Missing values in rows: {set(nan_rows)}")
if len(nan_cols) > 0:
    print(f"Missing values in columns: {set(nan_cols)}")
if len(inf_rows) > 0:
    print(f"Infinite values in rows: {set(inf_rows)}")
if len(inf_cols) > 0:
    print(f"Infinite values in columns: {set(inf_cols)}")

In [200]:
subset = dataset.dropna()
X = subset[factors]
X = sm.add_constant(X)

y = subset[stocks].sub(subset['RF'], axis=0)

betas = pd.DataFrame(index=stocks, columns=factors)
alphas = pd.DataFrame(index=stocks, columns=['Alpha'])

In [215]:
for stock in stocks:
    model = sm.OLS(y[stock], X).fit()
    betas.loc[stock] = model.params[factors]
    alphas.loc[stock] = model.params['const']

sub_return = pd.DataFrame(np.dot(ff4[factors],betas.T), index=ff4.index, columns=betas.index)
merge_return = pd.merge(sub_return,ff4['RF'], left_index=True, right_index=True)
daily_expected_returns = merge_return.add(merge_return['RF'],axis=0).drop('RF',axis=1).add(alphas.T.loc['Alpha'], axis=1)

expected_annual_return = ((daily_expected_returns+1).cumprod().tail(1) ** (1/daily_expected_returns.shape[0]) - 1) * 255
covariance_matrix = dataset[stocks].cov() * 255

In [232]:
def efficient_portfolio(returns, rf_rate, cov_matrix):
    num_assets = returns.shape[0] if len(returns.shape) == 1 else returns.shape[1]
    
    def neg_sharpe_ratio(weights):
        port_return = np.sum(returns * weights)
        port_std_dev = np.sqrt(weights @ cov_matrix @ weights.T)
        sharpe_ratio = (port_return - rf_rate) / port_std_dev
        return -sharpe_ratio
    
    constraints = [{'type': 'eq', 'fun': lambda w: np.sum(w) - 1},
                   {'type': 'ineq', 'fun': lambda w: w}]
    
    bounds = [(0, 1) for _ in range(num_assets)]
    init_weights = np.ones(num_assets) / num_assets
    opt_result = minimize(neg_sharpe_ratio, init_weights, method='SLSQP', bounds=bounds, constraints=constraints)
    
    opt_weights = opt_result.x * 100
    opt_port_return = np.sum(returns * opt_weights)
    opt_port_std_dev = np.sqrt(opt_weights @ cov_matrix @ opt_weights.T)
    opt_sharpe_ratio = (opt_port_return - rf_rate) / opt_port_std_dev
    return opt_weights, opt_sharpe_ratio

In [235]:
weight, sharpe_ratio = efficient_portfolio(expected_annual_return.values[0], 0.0425, covariance_matrix)
weight = pd.DataFrame(weights, columns=['weight %'])
display(weight)
print("The Portfolio's Sharpe Ratio is: ",sharpe_ratio)

,weight %
0,4.307682e-14
1,0.000000e+00
2,2.439030e+01
3,1.623790e-15
4,3.752205e-14
5,0.000000e+00
6,0.000000e+00
7,8.156784e-15
8,2.182235e-14
9,0.000000e+00


The Portfolio's Sharpe Ratio is:  1.7494940726026273


In [ ]:
opt_weights = pd.DataFrame(opt_weights, columns=['weights(%)'])
opt_weights['weights(%)'] = round(opt_weights*100, 2)